# SparkSubmitOperator

Acum că avem și tabelul putem să introducem date în acesta. Pentru a face asta, întâi trebuie să procesăm datele respective. Pentru a procesa datele o să folosim Spark (tot de la Apache, precum Airflow). Airflow este un sistem de orchestrare, nu unul de procesare a datelor, prin urmare nu trebuie să procesăm datele pe serverul de Ariflow. Ca să facem această procesare o să ne folosim de un cluster de Spark (creat și rulat când am porit toate aplicațiile). Ca să realizăm această procesare o să ne folosim de operatorul SparkSubmitOperator. Acesta are nevoie de următoarele:

- application = fișierul care să fie rulat în Spark

- conn_id = conexiunea către Spark

- verbose (opțional) = ca să nu apară multe loguri din Spark

Aplicația care o să ruleze pe spark o să fie salvată într-un fișier separat de python. Codul este următorul:

In [ ]:
from os.path import expanduser, join, abspath

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json

warehouse_location = abspath('spark-warehouse')

# Initialize Spark Session
spark = SparkSession \
    .builder \
    .appName("Forex processing") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

# Read the file forex_rates.json from the HDFS
df = spark.read.json('hdfs://namenode:9000/forex/forex_rates.json')

# Drop the duplicated rows based on the base and last_update columns
forex_rates = df.select('base', 'last_update', 'rates.eur', 'rates.usd', 'rates.cad', 'rates.gbp', 'rates.jpy', 'rates.nzd') \
    .dropDuplicates(['base', 'last_update']) \
    .fillna(0, subset=['EUR', 'USD', 'JPY', 'CAD', 'GBP', 'NZD'])

# Export the dataframe into the Hive table forex_rates
forex_rates.write.mode("append").insertInto("forex_rates")

În cadrul fișierului de mai sus este codul care o să ruleze utilizând Spark. Codul respectiv o să citească fișierul de date forex_rates.json (pe care l-am adăugat în HDFS) și o să facă drop la elementele care sunt duplicate. După ce se realizează această procesare o să fie introduse datele în tabelul forex_rates (tabel realizat cu operatorul HiveOperator)

Acum că avem codul de procesare din Spark, trebuie să ne creem și conexiunea către clusterul de Spark.

<img src="../../ss/airflow-section-03/section-03-ss-11.png">

Conexinea din Spark este următoarea. După cum spuneam, clusterul de Spark a fost creat cu fișierul acela de docker-compose, iar datele respective au fost trecute în acel fișier (datele pentru host și port).

Acum că avem datele putem să ne creem acest operator din Spark pentru procesare.

In [ ]:
from airflow.providers.apache.spark.operators.spark_submit import SparkSubmitOperator

process_forex_rates_spark = SparkSubmitOperator(
    task_id="process_forex_rates_spark",
    conn_id="spark_conn",
    application="/opt/airflow/dags/scripts/forex_processing.py",
    verbose=False
)

Ca să fie complet codul, trebuie doar să adaugăm acest cod în cadrul fișierului cu DAG-ul. Ca să verificăm dacă acest cod a fost rulat, putem să verificăm dacă există valori adăugate în cadrul tabelului pe care l-am creat anterior. Pentru a face asta o să ne folosim din nou de HUE, de partea de HQL unde o să rulăm comanda "SELECT * FROM forex_rates"

În HUE trebuie să vedem următorul rezultat

<img src="../../ss/airflow-section-03/section-03-ss-12.png">

După cum se poate observa, există date în cadrul acestui tabel, ceea ce înseamnă că procesare prin Spark a funcționat.